In [1]:
# Selenium 라이브러리 설치
!pip install selenium
from selenium import webdriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [2]:
# 패키지와 드라이버 설치
import sys
!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/

PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/saiarcot895/chromium-beta/ubuntu/ jammy main'
Description:
This PPA contains the latest Chromium Beta builds, with hardware video decoding enabled (hidden behind a flag), and support for Widevine (needed for viewing many DRM-protected videos) enabled.

== Hardware Video Decoding ==

To enable hardware video decoding, start Chromium with the --enable-features=VaapiVideoDecoder argument. To make this persistent, create a file at /etc/chromium-browser/customizations/92-vaapi-hardware-decoding with the following contents:

CHROMIUM_FLAGS="${CHROMIUM_FLAGS} --enable-features=VaapiVideoDecoder"

See also https://wiki.archlinux.org/title/Chromium#Hardware_video_acceleration for more information on VAAPI video decoding support.

=== Widevine Support ===

The packages in this PPA have support for Widevine inside Chromium enabled. However, you still need to copy some files from 

In [35]:
# Selenium 관련 환경 설정
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, InvalidSessionIdException, ElementClickInterceptedException
from time import sleep
from bs4 import BeautifulSoup
import re
import json

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('disable-dev-shm-usage')
options.add_experimental_option("detach", True)

webdriver_service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(service=webdriver_service, options=options)

In [36]:
# 웹페이지 파싱
url = "https://map.naver.com/p/entry/place/1915239189?c=15.16,0,0,0,dh&placePath=/review"
driver.get(url)

res = driver.page_source
soup = BeautifulSoup(res, 'html.parser')
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="2d8c183d1f888682eac181409ff0db6a")>


In [5]:
# frame 변경 메소드
def switch_frame(frame):
  driver.switch_to.default_content()  # frame 초기화
  sleep(1)
  driver.switch_to.frame(frame)  # frame 변경
  res
  soup

In [6]:
# css 찾을때 까지 10초 대기
def time_wait(num, code):
  try:
    wait = WebDriverWait(driver, num).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, code)))
  except:
    print(code, '태그를 찾지 못하였습니다.')
    driver.quit()
  return wait

In [37]:
switch_frame("entryIframe")
sleep(5)

start = time.time()
review_dict = []
print('[크롤링 시작...]')

try:
    for _ in range(46):
        try:
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            sleep(3)
            more_button = driver.find_element(By.CSS_SELECTOR, '.fvwqf')
            driver.execute_script("arguments[0].click();", more_button)
            print('더보기<<')
            sleep(3)
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)

            '''more_button = WebDriverWait(driver, 20).until(
                              EC.element_to_be_clickable((By.CSS_SELECTOR, '.fvwqf')))
            try:
                more_button.click()
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", more_button)'''

        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
            break

    review_list = WebDriverWait(driver, 10).until(
                  EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.owAeM')))

    # 각 리뷰 텍스트를 가져와서 리스트에 추가
    all_keyword = []
    cnt = 0
    for review in review_list:
        cnt += 1
        try:
            keyword_button = review.find_element(By.CSS_SELECTOR, 'a.sIv5s.WPk67')
            keyword_button.click()
            time.sleep(2)

            keyword1 = review.find_elements(By.CSS_SELECTOR, 'span.sIv5s')[0].text
            keyword2 = review.find_elements(By.CSS_SELECTOR, 'span.sIv5s')[1].text
            time.sleep(2)

            text = review.find_element(By.CSS_SELECTOR, 'zPfVt').text

        except:
            n = len(review.find_elements(By.CSS_SELECTOR, 'span.sIv5s'))
            keyword1 = review.find_elements(By.CSS_SELECTOR, 'span.sIv5s')[0].text if n>0 else ' '
            keyword2 = review.find_elements(By.CSS_SELECTOR, 'span.sIv5s')[1].text if n>1 else ' '

            try:
                toggle_button = review.find_element(By.CSS_SELECTOR, '.rvCSr')
                driver.execute_script("arguments[0].click();", toggle_button)
                time.sleep(2)
                text = review.find_element(By.CSS_SELECTOR, '.zPfVt').text

            except NoSuchElementException:
                text = review.find_element(By.CSS_SELECTOR, '.zPfVt').text

        dict_temp = {'1st_keyword':keyword1,
                      '2ed_keyword':keyword2,
                      'review':text
                     }
        review_dict.append(dict_temp)
        print(f'{cnt}번째 리뷰 ...완료')

except Exception as e:
    print(f"ERROR: {e}")

finally:
    print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
    driver.quit()

[크롤링 시작...]
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
더보기<<
-더보기 버튼 모두 클릭 완료-
1번째 리뷰 ...완료
2번째 리뷰 ...완료
3번째 리뷰 ...완료
4번째 리뷰 ...완료
5번째 리뷰 ...완료
6번째 리뷰 ...완료
7번째 리뷰 ...완료
8번째 리뷰 ...완료
9번째 리뷰 ...완료
10번째 리뷰 ...완료
11번째 리뷰 ...완료
12번째 리뷰 ...완료
13번째 리뷰 ...완료
14번째 리뷰 ...완료
15번째 리뷰 ...완료
16번째 리뷰 ...완료
17번째 리뷰 ...완료
18번째 리뷰 ...완료
19번째 리뷰 ...완료
20번째 리뷰 ...완료
21번째 리뷰 ...완료
22번째 리뷰 ...완료
23번째 리뷰 ...완료
24번째 리뷰 ...완료
25번째 리뷰 ...완료
26번째 리뷰 ...완료
27번째 리뷰 ...완료
28번째 리뷰 ...완료
29번째 리뷰 ...완료
30번째 리뷰 ...완료
31번째 리뷰 ...완료
32번째 리뷰 ...완료
33번째 리뷰 ...완료
34번째 리뷰 ...완료
35번째 리뷰 ...완료
36번째 리뷰 ...완료
37번째 리뷰 ...완료
38번째 리뷰 ...완료
39번째 리뷰 ...완료
40번째 리뷰 ...완료
41번째 리뷰 ...완료
42번째 리뷰 ...완료
43번째 리뷰 ...완료
44번째 리뷰 ...완료
45번째 리뷰 ...완료
46번째 리뷰 ...완료
47번째 리뷰 ...완료
48번째 리뷰 ...완료
49번째 리뷰 ...완료
50번째 리뷰 ...완료
51번째 리뷰 ...완료
52번째 리뷰 ...완료
53번째 리뷰 ...완료
54번째 리뷰 ...완료
55번째 리뷰 ...완료
56번째 리뷰 ...완료
57번째 리뷰 ...완료
58번째 리뷰 ...완료
59번째 

In [38]:
with open('/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/review_data.json', 'w', encoding='utf-8') as f:
    json.dump(review_dict, f, indent=4, ensure_ascii=False)

In [39]:
import json

# JSON 파일 경로 설정
file_path = '/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/review_data.json'

# JSON 파일 열기
with open(file_path, 'r') as file:
    data = json.load(file)
data

[{'1st_keyword': '샐러드바가 잘 되어있어요',
  '2ed_keyword': '혼밥하기 좋아요',
  'review': '❥ menu : 육회덮밥 (11,000)\n\n❥ 모미지식당에서 저는 육회덮밥만 먹어요. 육회가 질기지 않고 부드러워요. 밥이랑 같이 먹으면 짜지도 않고 간도 딱 좋아요 ♡ 이대 맛집 인정 ♡ \n\n❥ 셀프바에 있는 파인애플은 설탕뿌린 것처럼 항상 달달하고 맛있어요! 그리고 샐러드도 장국도 맛있어서 그냥 완벽하다는 생각밖에 안 듭니다.. \n\n❥ 테이블마다 키오스크가 있고 분할 계산이 돼서 친구랑 갔을 때도 각자 계산하기 편해요 ! \n\n방학때도 자주 갈게요 ~'},
 {'1st_keyword': '혼밥하기 좋아요',
  '2ed_keyword': '음식이 맛있어요',
  'review': '가게 외관상으로는 \n모르겠지만 리뷰 평점이 높아서 \n가보았던 모미지 식당 \n\n왜 평점이 높은지 요소들이 보였다.\n1.직접 이용할 수 있는 반찬+장국 셀프바 \n2.혼밥+ 데이트 하기 좋은 분위기 + 자리마다 주문용 테이블 있음(선불결제) \n\n3. 음식 자체가 맛있고 목넘김이 편안한 음식들 \n\n육회덮밥 = 육회가 질기지 않고 잡내 안나며 노른자 터트려서 밥이랑 먹기 좋음 + 느끼하면 와사비 사알짝 곁들여 먹기 \n소고기가지덮밥 = 가지가 거슬리게 씹히지 않고 고기와 부드럽게 씹힘 + 김치/단무지 와 함께 하기 좋음 \n모듬 고로케 = 야채+고구마치즈+카레 조합으로 잘 찾을 것 같은 세가지 조합으로 해둠 \n샐러드 = 흑임자 드레싱으로 위의 음식들과 잘 어울림 \n장국 = 미소장국 인데 시큼한 맛이 안남(고소+담백)'},
 {'1st_keyword': '음식이 맛있어요',
  '2ed_keyword': '양이 많아요',
  'review': '진짜 맛있습니다 음식도 주문하면 바로바로 나옵니다 그리고 시그니쳐 소고기가지덮밥은 진짜 강추입니다'},
 {'1st_keyword': '음식이 맛있어요',
  '2ed_keyword

In [40]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv("/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/review_data.csv", index=False)

In [42]:
df.head()

,1st_keyword,2ed_keyword,review
0,샐러드바가 잘 되어있어요,혼밥하기 좋아요,"❥ menu : 육회덮밥 (11,000)\n\n❥ 모미지식당에서 저는 육회덮밥만 먹..."
1,혼밥하기 좋아요,음식이 맛있어요,가게 외관상으로는 \n모르겠지만 리뷰 평점이 높아서 \n가보았던 모미지 식당 \n\...
2,음식이 맛있어요,양이 많아요,진짜 맛있습니다 음식도 주문하면 바로바로 나옵니다 그리고 시그니쳐 소고기가지덮밥은 ...
3,음식이 맛있어요,재료가 신선해요,가지덮밥 전문으로 하는 곳이라 방문해봤는데\n벌써 3번째 방문하고 드디어 후기남깁니...
4,음식이 맛있어요,특별한 메뉴가 있어요,이대생이라면 모두가 아는 맛집이죠ㅎㅎㅎ 육회덮밥 늘 맛있어요! 타대생 친구나 손님 ...
